# PySpark example with linear regression model

This set of notes covers the estimation and prediction with a linear regression model, and the use of PySpark in order to parallelize and scale in computation.  In addition, the notes provide a brief commentary on privacy preserving machine learning models.

## Linear regression refresher

To fit a linear regression (without any penalty or regularization) can be done via the least-squares estimation:

$$\min_\beta~\ell(\beta) = \|\mathbf{y} - \mathbf{X} \beta \|_2^2,$$

The corresponding analytical solution can be found by setting the partial derivatives to zero:
$$\frac{\partial \ell(\beta)}{\partial \beta} \equiv 0 \qquad \Rightarrow \qquad \widehat{\beta} = (\mathbf{X}^\mathsf{T}\mathbf{X})^{-1} \mathbf{X}^\mathsf{T}\mathbf{y}.$$

*Why do we need to think about parallelization at all?*

Say we have $n$ data points, i.e. $(\mathbf{x}_i, y_i)_{i=1}^n$. Take a look at the dimension of the quantities of interest:
$$\mathbf{X} \in \mathbb{R}^{n \times (d+1)}, \mathbf{y} \in \mathbb{R}^n.$$

The major computations involve three operations:  
1. Multiplication: $\mathbf{X}^\mathsf{T}\mathbf{X} = \sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^\mathsf{T},$
2. Multiplication: $\mathbf{X}^\mathsf{T}\mathbf{y} = \sum_{i=1}^n \mathbf{x_i} y_i,$
3. Inversion of matrix: $(\mathbf{X}^\mathsf{T}\mathbf{X})^{-1} \mathbf{X}^\mathsf{T}\mathbf{y}.$

*Rationale:*  
- In the case where $n$ is extremely large, the multiplication and summation, steps (1) and (2), are slow.
- Since the inversion of matrix is driven by the size of $\mathbf{X}^\mathsf{T}\mathbf{X}$, which is typically a relatively small $(d+1) \times (d+1)$ matrix.  Step (3) is relatively cheap in terms of computation.

In [1]:
# You typically do not need this

import os
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-11.0.2"
os.environ["SPARK_HOME"] = r"C:\Program Files\Spark\spark-3.5.5-bin-hadoop3"

import findspark
findspark.init()

## Diamonds dataset

In [2]:
import seaborn as sns
diam = sns.load_dataset('diamonds', cache=True, data_home='dataset/')

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

diamonds = (
    spark.read.format('csv')
    .options(header='true', inferSchema='true')
    .load('dataset/diamonds.csv')
    .cache()
)

In [4]:
diamonds.take(5)

[Row(carat=0.23, cut='Ideal', color='E', clarity='SI2', depth=61.5, table=55.0, price=326, x=3.95, y=3.98, z=2.43),
 Row(carat=0.21, cut='Premium', color='E', clarity='SI1', depth=59.8, table=61.0, price=326, x=3.89, y=3.84, z=2.31),
 Row(carat=0.23, cut='Good', color='E', clarity='VS1', depth=56.9, table=65.0, price=327, x=4.05, y=4.07, z=2.31),
 Row(carat=0.29, cut='Premium', color='I', clarity='VS2', depth=62.4, table=58.0, price=334, x=4.2, y=4.23, z=2.63),
 Row(carat=0.31, cut='Good', color='J', clarity='SI2', depth=63.3, table=58.0, price=335, x=4.34, y=4.35, z=2.75)]

In [5]:
df = (
    diamonds
    .where(diamonds['price'] > 1000)
    .select(['cut', 'color', 'carat', 'clarity', 'price'])
)

In [6]:
df.show(5)

+---------+-----+-----+-------+-----+
|      cut|color|carat|clarity|price|
+---------+-----+-----+-------+-----+
|    Ideal|    E|  0.7|    SI1| 2757|
|     Fair|    E| 0.86|    SI2| 2757|
|    Ideal|    G|  0.7|    VS2| 2757|
|Very Good|    E| 0.71|    VS2| 2759|
|Very Good|    G| 0.78|    SI2| 2759|
+---------+-----+-----+-------+-----+
only showing top 5 rows



In [9]:
row = diamonds.take(1)

In [7]:
predictors = ['cut', 'color', 'carat', 'clarity']
categorical = set(['cut', 'color', 'clarity'])

In [8]:
response = 'price'

In [15]:
def xtx_map(row):
    row = row.asDict()
    for i in predictors:
        (ki, vi) = (i, row[i]) if i not in categorical else (i+"_"+row[i], 1.0)
        for j in predictors:
            (kj, vj) = (j, row[j]) if j not in categorical else (j+"_"+row[j], 1.0)
            yield ((ki, kj), vi * vj)

def xty_map(row):
    row = row.asDict()
    for j in predictors:
        (kj, vj) = (j, row[j]) if j not in categorical else (j+"_"+row[j], 1.0)
        yield (kj, vj * row[response])

In [13]:
row = df.take(1)
row[0]

Row(cut='Ideal', color='E', carat=0.7, clarity='SI1', price=2757)

In [16]:
[a for a in xty_map(row[0])]

[('cut_Ideal', 2757.0),
 ('color_E', 2757.0),
 ('carat', 1929.8999999999999),
 ('clarity_SI1', 2757.0)]

In [17]:
[a for a in xtx_map(row[0])]

[(('cut_Ideal', 'cut_Ideal'), 1.0),
 (('cut_Ideal', 'color_E'), 1.0),
 (('cut_Ideal', 'carat'), 0.7),
 (('cut_Ideal', 'clarity_SI1'), 1.0),
 (('color_E', 'cut_Ideal'), 1.0),
 (('color_E', 'color_E'), 1.0),
 (('color_E', 'carat'), 0.7),
 (('color_E', 'clarity_SI1'), 1.0),
 (('carat', 'cut_Ideal'), 0.7),
 (('carat', 'color_E'), 0.7),
 (('carat', 'carat'), 0.48999999999999994),
 (('carat', 'clarity_SI1'), 0.7),
 (('clarity_SI1', 'cut_Ideal'), 1.0),
 (('clarity_SI1', 'color_E'), 1.0),
 (('clarity_SI1', 'carat'), 0.7),
 (('clarity_SI1', 'clarity_SI1'), 1.0)]

In [18]:
xtx_data = (df.rdd
            .flatMap(xtx_map)
            .reduceByKey(lambda a, b: a+b)
            .collect()
           )

In [23]:
xty_data = (df.rdd
            .flatMap(xty_map)
            .reduceByKey(lambda a, b: a+b)
            .collect()
           )

In [24]:
xty_data

[('cut_Ideal', 69491685.0),
 ('color_E', 27913897.0),
 ('carat', 259765355.250002),
 ('clarity_SI1', 50141077.0),
 ('cut_Fair', 6931384.0),
 ('clarity_SI2', 45876510.0),
 ('color_G', 42841867.0),
 ('clarity_VS2', 45536589.0),
 ('cut_Very Good', 45996850.0),
 ('cut_Good', 18558296.0),
 ('color_F', 33724244.0),
 ('clarity_VS1', 29642262.0),
 ('cut_Premium', 60868420.0),
 ('color_H', 35873113.0),
 ('color_D', 19990260.0),
 ('color_I', 26838093.0),
 ('clarity_VVS2', 15250346.0),
 ('clarity_VVS1', 8043137.0),
 ('color_J', 14665161.0),
 ('clarity_I1', 2860076.0),
 ('clarity_IF', 4496638.0)]

In [25]:
index = dict(zip([r[0] for r in xty_data], range(len(xty_data))))
p = len(index)

index

{'cut_Ideal': 0,
 'color_E': 1,
 'carat': 2,
 'clarity_SI1': 3,
 'cut_Fair': 4,
 'clarity_SI2': 5,
 'color_G': 6,
 'clarity_VS2': 7,
 'cut_Very Good': 8,
 'cut_Good': 9,
 'color_F': 10,
 'clarity_VS1': 11,
 'cut_Premium': 12,
 'color_H': 13,
 'color_D': 14,
 'color_I': 15,
 'clarity_VVS2': 16,
 'clarity_VVS1': 17,
 'color_J': 18,
 'clarity_I1': 19,
 'clarity_IF': 20}

In [26]:
# arrange the individual elements back into matrices
import numpy as np

XTY = np.zeros((p, 1))
for (k, v) in xty_data:
  XTY[index[k]] = v

XTX = np.zeros((p,p))
for ((k1,k2),v) in xtx_data:
  XTX[index[k1], index[k2]] = v

In [27]:
XTX.shape, XTY.shape

((21, 21), (21, 1))

In [28]:
beta = np.linalg.solve(XTX, XTY)

In [29]:
beta

array([[-3135410.6050034 ],
       [ 4659378.08817701],
       [    9805.68171107],
       [-1527908.27694775],
       [-3136368.13604856],
       [-1528947.23087084],
       [ 4658923.0870537 ],
       [-1527130.54041287],
       [-3135608.232303  ],
       [-3135866.36037277],
       [ 4659253.08401101],
       [-1526746.38065197],
       [-3135668.49669144],
       [ 4658246.93221199],
       [ 4659592.70335286],
       [ 4657612.83283337],
       [-1526110.26492251],
       [-1525873.9198634 ],
       [ 4656625.8293952 ],
       [-1531846.19242137],
       [-1525319.83283582]])